In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

dataset: [link](https://dados.gov.br/dados/conjuntos-dados/gestao-de-pessoas-executivo-federal--aposentados)

# Motivação
O envelhecimento da força de trabalho no serviço público federal impacta diretamente os custos previdenciários e a reposição de profissionais. Entender padrões nas aposentadorias (como cargos, órgãos e tempo de serviço) pode ajudar gestores públicos a planejar políticas de recursos humanos e sustentabilidade fiscal.

# Hipótese
É possível prever o valor do rendimento líquido de um aposentado com base em suas características funcionais e tipo de aposentadoria?

In [13]:
df = pd.read_csv('../dataset/APOSENTADOS_112024.csv', delimiter=';', encoding='latin1', header=None)
df.columns = [
    "Nome",
    "CPF",
    "Matricula",
    "Nome_Orgao",
    "Sigla_Orgao",
    "Codigo_Orgao_Superior",
    "Cargo",
    "Classe",
    "Padrao",
    "Referencia",
    "Nivel",
    "Tipo_Aposentadoria",
    "Fundamentacao_Inatividade",
    "Nome_Diploma_Legal",
    "Data_Publicacao_Diploma_Legal",
    "Ocorrencia_Ingresso_Servico_Publico",
    "Data_Ingresso_Servico_Publico",
    "Valor_Rendimento_Liquido"
]

df.head()

,Nome,CPF,Matricula,Nome_Orgao,Sigla_Orgao,Codigo_Orgao_Superior,Cargo,Classe,Padrao,Referencia,Nivel,Tipo_Aposentadoria,Fundamentacao_Inatividade,Nome_Diploma_Legal,Data_Publicacao_Diploma_Legal,Ocorrencia_Ingresso_Servico_Publico,Data_Ingresso_Servico_Publico,Valor_Rendimento_Liquido
0,GIUSEPPE DE CASSIO MACHADO MODICAMORE ...,***343511**,133000000895,MINISTERIO DA AGRICULTURA E PECUARIA,MAPA,0,AGENTE DE PORTARIA,S,III,0,0,INCAPACIDADE,EC103 10 Ý1Ì II - INCAPACIDADE,...,9102024,ADMISSAO SEM CONCURSO PUBLICO ...,12111981,"3.135,44"
1,VALDECINO ANTONIO DA COSTA ...,***745701**,133000001411,MINISTERIO DA AGRICULTURA E PECUARIA,MAPA,0,AGENTE DE VIGILANCIA,S,III,0,0,VOLUNTARIA,EC103 3Ì EC47 3Ì,...,1102024,ADMISSAO POR CONCURSO PUBLICO ...,22021985,"7.200,67"
2,JOAO CARLOS TEIXEIRA ...,***769607**,133000004402,MINISTERIO DA AGRICULTURA E PECUARIA,MAPA,0,AGENTE ADMINISTRATIVO,S,III,0,0,VOLUNTARIA,EC103 3Ì EC47 3Ì,...,10102024,ADMISSAO POR CONCURSO PUBLICO ...,1021983,"4.094,32"
3,JORGE HERALDO SALES DO NASCIMENTO ...,***686572**,133000009075,MINISTERIO DA AGRICULTURA E PECUARIA,MAPA,0,AGENTE DE INSP SANIT IND PROD ORIGEM ANI,S,IV,0,0,VOLUNTARIA,EC103 3Ì EC47 3Ì,...,1102024,ADMISSAO SEM CONCURSO PUBLICO ...,30031983,"7.721,18"
4,FRANCISCO LOPES DE MATTOS ...,***893649**,133000010213,MINISTERIO DA AGRICULTURA E PECUARIA,MAPA,0,AGENTE DE INSP SANIT IND PROD ORIGEM ANI,S,IV,0,0,VOLUNTARIA,EC103 3Ì EC47 3Ì,...,31102024,DECISAO JUDICIAL ...,1101978,"8.753,64"


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687 entries, 0 to 686
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Nome                                 687 non-null    object
 1   CPF                                  687 non-null    object
 2   Matricula                            687 non-null    int64 
 3   Nome_Orgao                           687 non-null    object
 4   Sigla_Orgao                          687 non-null    object
 5   Codigo_Orgao_Superior                687 non-null    int64 
 6   Cargo                                687 non-null    object
 7   Classe                               687 non-null    object
 8   Padrao                               687 non-null    object
 9   Referencia                           687 non-null    int64 
 10  Nivel                                687 non-null    int64 
 11  Tipo_Aposentadoria                   687 non-

In [12]:
df.isnull().sum()

Nome                                   0
CPF                                    0
Matricula                              0
Nome_Orgao                             0
Sigla_Orgao                            0
Codigo_Orgao_Superior                  0
Cargo                                  0
Classe                                 0
Padrao                                 0
Referencia                             0
Nivel                                  0
Tipo_Aposentadoria                     0
Fundamentacao_Inatividade              0
Nome_Diploma_Legal                     0
Data_Publicacao_Diploma_Legal          0
Ocorrencia_Ingresso_Servico_Publico    0
Data_Ingresso_Servico_Publico          0
Valor_Rendimento_Liquido               0
dtype: int64